# USGS API Shakemap

This Jupyter notebook will walk through how to access an USGS Shakemap API. The goal of this example is to use an API request to retrieve a USGS Shakemap (https://earthquake.usgs.gov/data/shakemap/) and plot the shakemap for the earthquake using a Python Package called Folium (https://python-visualization.github.io/folium/)
 
 
## Requirements
folium  
geopandas  
requests  
json  
numpy  
pandas

## Install packages
To begin building, we will install first install the Folium Python Package which we will use later on. To install folium, we will use the "pip install" framework

In [2]:
pip install -q folium

Note: you may need to restart the kernel to use updated packages.


## Import packages
Next to set-up the notebook we will call various packages and modules using the " import function". This will allow us to more seamlessly use the packages as needed throughout the notebook. 

In [3]:
import requests
import numpy as np
import json
import pandas as pd
import folium


## API Data Call
A url request is made to the USGS website, to download shakemap contours. Users can navigate to any specific event of their liking and use the url in the "Download data" section of the event. For this example, we will be looking at the Mw=4.2 earthquake off the coast of Malibu (https://earthquake.usgs.gov/earthquakes/eventpage/ci40161279/executive). "Get requests" is used to pull the url information, and is then saved into a variable with a JSON Format. 

In [4]:
url = 'https://earthquake.usgs.gov/product/shakemap/40161279/ci/1675464767472/download/cont_pga.json'
r = requests.get(url)
json_data= r.json()

## GeoJSON output 

To understand the GEOJSON output from the URL request you can look to the USGS summary here: https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php. The main points to include are that the each earthquake has its own features. For our case, we are looking at Peak Ground Intensity contours, and therefore each "feature" is for a different PGA interval. The interval is indicated in the "value" and is a nested key under features. Within the "features" there are also coordinates for the pga contours that we will use to recreate the shake map. 

## Map Creation 

To create a map, we will utilize a for loop within Folium. We first initialize our map, m with a general latitude and longitude. We also use the tile "Stamen Terrain" as our base layer for our map. We loop through the json_data via the different features, and plot the differnet PGA contours. We need to flip the coordinates, to ensure the values are plotted correctly, and the "polyline" function is used to connect the contours. Lasktly a marker is added to each polyline to denote the PGA value. 

In [6]:
from folium.features import DivIcon

m=folium.Map(locations=[40.525,-124.423],zoom_start=25,control_scale=True,tiles= 'Stamen Terrain',
                        attr='ESRI')

# Bounds for contiguous US - starting bounds for map
map_bounds = (
    (35.87036874083626, -120.7759234053426), (32.560670391680134, -115.87929177039352)
)
m.fit_bounds(map_bounds)

for feature in json_data['features']:
   
    pga = feature['properties']['value']

    for shape_data in feature['geometry']['coordinates']:
        shape = np.flip(np.array(shape_data).reshape(-1, 2), (0, 1))
        folium.PolyLine(shape,color='#E97025',weight=5, opacity=0.8).add_to(m)
        
        first_point = shape[0]
        
        folium.map.Marker(first_point,
          icon=DivIcon(
              icon_size=(30,30),
              icon_anchor=(5,14),
              html=f'<div style="font-size: 14pt">%s</div>' % str(pga),
          )
         ).add_to(m)
m